In [1]:
import pandas as pd

In [2]:
repository = pd.read_csv("./autos.csv", encoding='ISO-8859-1')

In [3]:
repository = repository.drop('dateCrawled', axis=1)\
                       .drop('dateCreated', axis=1)\
                       .drop('nrOfPictures', axis=1)\
                       .drop('postalCode', axis=1)\
                       .drop('name', axis=1)\
                       .drop('seller', axis=1)\
                       .drop('offerType', axis=1)\
                       .drop('lastSeen', axis=1)

In [4]:
i1 = repository.loc[repository.price <= 10]
repository = repository[repository.price > 10]

In [5]:
i1 = repository.loc[repository.price >= 350000]
repository = repository[repository.price < 350000]

In [6]:
repository.loc[pd.isnull(repository['vehicleType'])]
repository['vehicleType'].value_counts() #limousine

limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: vehicleType, dtype: int64

In [7]:
repository.loc[pd.isnull(repository['gearbox'])]
repository['gearbox'].value_counts() # manuell

manuell      266547
automatik     75508
Name: gearbox, dtype: int64

In [8]:
repository.loc[pd.isnull(repository['model'])]
repository['model'].value_counts() # golf

golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: model, Length: 251, dtype: int64

In [9]:
repository.loc[pd.isnull(repository['fuelType'])]
repository['fuelType'].value_counts() # benzin

benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: fuelType, dtype: int64

In [10]:
repository.loc[pd.isnull(repository['notRepairedDamage'])]
repository['notRepairedDamage'].value_counts() # nein

nein    259301
ja       34004
Name: notRepairedDamage, dtype: int64

In [11]:
values = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}

repository = repository.fillna(value=values)

In [12]:
previsors = repository.iloc[:, 1:13].values
real_price = repository.iloc[:, 0].values


In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelEncoder_prev = LabelEncoder()
previsors[:,0] = labelEncoder_prev.fit_transform(previsors[:,0])
previsors[:,1] = labelEncoder_prev.fit_transform(previsors[:,1])
previsors[:,3] = labelEncoder_prev.fit_transform(previsors[:,3])
previsors[:,5] = labelEncoder_prev.fit_transform(previsors[:,5])
previsors[:,8] = labelEncoder_prev.fit_transform(previsors[:,8])
previsors[:,9] = labelEncoder_prev.fit_transform(previsors[:,9])
previsors[:,10] = labelEncoder_prev.fit_transform(previsors[:,10])

In [14]:
import numpy as np
encoder = OneHotEncoder(sparse=False)
cols_to_encode = [0, 1, 3, 5, 8, 9, 10]
columns_to_encode = previsors[:, cols_to_encode]
encoded_columns = encoder.fit_transform(columns_to_encode)
dataset_encoded = np.delete(previsors, cols_to_encode, axis=1)
dataset_encoded = np.concatenate((dataset_encoded, encoded_columns), axis=1)
previsorsEncoded = dataset_encoded.astype(float)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [15]:
previsorsEncoded

array([[1.993e+03, 0.000e+00, 1.500e+05, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [2.011e+03, 1.900e+02, 1.250e+05, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.004e+03, 1.630e+02, 1.250e+05, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       ...,
       [1.996e+03, 1.020e+02, 1.500e+05, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [2.002e+03, 1.000e+02, 1.500e+05, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [2.013e+03, 3.200e+02, 5.000e+04, ..., 0.000e+00, 0.000e+00,
        1.000e+00]])

In [16]:
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor

In [17]:
def createNetwork():
    regressor = Sequential()
    regressor.add(Dense(units=158, activation='relu', input_dim=316))
    regressor.add(Dense(units=158, activation='relu'))
    regressor.add(Dense(units=1, activation='linear'))
    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

    return regressor

In [18]:
regressor = KerasRegressor(build_fn=createNetwork, epochs=100, batch_size=300)
results = cross_val_score(estimator=regressor, X=previsorsEncoded, y=-real_price, cv=10, scoring='neg_mean_absolute_error',  n_jobs=-1)

C:\Users\Greze\AppData\Local\Temp\ipykernel_16204\2938390706.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  regressor = KerasRegressor(build_fn=createNetwork, epochs=100, batch_size=300)


In [ ]:
mean = results.mean()
std = results.std()



11228/11228 [==============================] - 16s 1ms/step
